In [944]:
from pycoingecko import CoinGeckoAPI
from datetime import datetime, timedelta
import pandas as pd

In [945]:
# CoinGecko
cg = CoinGeckoAPI()

Se crea una instancia de la API de CoinGecko llamada cg para poder acceder a los datos y funciones proporcionados por `CoinGecko`.

-----------

In [946]:
# Mes y el año para filtrar
mes = 12
anio = 2021

Se establece el mes y el año para filtrar los datos de análisis.

------------

In [947]:
df_top_15_capitalizacion_actual = pd.read_csv("../data/top_15_capitalizacion_actual.csv")
lista_monedas = df_top_15_capitalizacion_actual['id'].tolist()
lista_monedas

['bitcoin',
 'ethereum',
 'tether',
 'binancecoin',
 'ripple',
 'usd-coin',
 'staked-ether',
 'cardano',
 'dogecoin',
 'solana',
 'tron',
 'polkadot',
 'matic-network',
 'litecoin',
 'shiba-inu']

Se lee un archivo CSV llamado `"top_15_capitalizacion_actual.csv"` que contiene los detalles de las 15 principales criptomonedas por capitalización de mercado actual. Luego, se extrae la columna `'id'` y se crea una lista llamada lista_monedas con los ids de las criptomonedas.

---------------

In [948]:
data = []

# Completa la lista
for moneda in lista_monedas:
    detalles = cg.get_coin_by_id(moneda)
    last_day_of_month = (datetime(anio, mes, 1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
    
    precios_historicos = cg.get_coin_market_chart_range_by_id(
        id=moneda,
        vs_currency='usd',
        from_timestamp=int(datetime(anio, mes, 1).timestamp()),
        to_timestamp=int(last_day_of_month.timestamp()))

    for i, punto in enumerate(precios_historicos['prices']):
        data.append({
            'fecha': datetime.utcfromtimestamp(punto[0] // 1000).strftime('%Y-%m'),
            'id': detalles['id'],
            'precio': punto[1],
            'ranking_capitalizacion': detalles['market_cap_rank'],
            'capitalizacion': detalles['market_data']['market_cap']['usd'],
            'volumen': precios_historicos['total_volumes'][i][1]
        })


1. Se inicializa una lista vacía llamada `data` para almacenar los datos recopilados.
2. Se itera sobre cada moneda en la lista `lista_monedas`.
3. Para cada moneda, se obtienen los detalles completos utilizando la función `cg.get_coin_by_id(moneda)` de la API de CoinGecko.
4. Se calcula el último día del mes utilizando la variable `last_day_of_month`, asegurándose de no exceder el último día válido del mes.
5. Se obtienen los precios históricos de la moneda en el rango de tiempo desde el primer día del mes hasta el último día del mes utilizando la función `cg.get_coin_market_chart_range_by_id()`.
6. Se itera sobre cada punto de precios históricos y se agregan los datos relevantes a la lista `data`, incluyendo la fecha, el ID de la criptomoneda, el precio, el ranking de capitalización, la capitalización de mercado y el volumen.

--------------

In [949]:
df = pd.DataFrame(data)

Se crea un DataFrame df a partir de la lista `data`, que contiene los datos recopilados de las criptomonedas.

In [950]:
df

,fecha,id,precio,ranking_capitalizacion,capitalizacion,volumen
0,2021-12,bitcoin,57483.820923,1,556198646739,3.576021e+10
1,2021-12,bitcoin,57365.336006,1,556198646739,3.568764e+10
2,2021-12,bitcoin,57153.808048,1,556198646739,3.570528e+10
3,2021-12,bitcoin,56825.208053,1,556198646739,3.481351e+10
4,2021-12,bitcoin,57034.263390,1,556198646739,3.468153e+10
...,...,...,...,...,...,...
10811,2021-12,shiba-inu,0.000034,15,5412777834,9.200908e+08
10812,2021-12,shiba-inu,0.000034,15,5412777834,9.191253e+08
10813,2021-12,shiba-inu,0.000034,15,5412777834,8.438848e+08
10814,2021-12,shiba-inu,0.000034,15,5412777834,8.150344e+08


----------

In [951]:
df['id'].unique()

array(['bitcoin', 'ethereum', 'tether', 'binancecoin', 'ripple',
       'usd-coin', 'staked-ether', 'cardano', 'dogecoin', 'solana',
       'tron', 'polkadot', 'matic-network', 'litecoin', 'shiba-inu'],
      dtype=object)

Se obtienen los ids únicos de las criptomonedas en el DataFrame `"df"`.

----------

In [952]:
df.shape

(10816, 6)

Se muestra la forma (número de filas y columnas) del DataFrame `"df"`.

----------

In [953]:
df.dropna(inplace=True)

Se eliminan las filas con valores faltantes (NaN) en el DataFrame `"df"`.

-------------

In [954]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10816 entries, 0 to 10815
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   fecha                   10816 non-null  object 
 1   id                      10816 non-null  object 
 2   precio                  10816 non-null  float64
 3   ranking_capitalizacion  10816 non-null  int64  
 4   capitalizacion          10816 non-null  int64  
 5   volumen                 10816 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 507.1+ KB


Se muestra información sobre el DataFrame df, incluyendo el tipo de datos de cada columna y la cantidad de valores no nulos.

---------------

In [955]:
df = df.groupby('id').agg({'precio': 'max', 'fecha': 'first', 'ranking_capitalizacion': 'min', 'capitalizacion': 'max', 'volumen': 'max'}).reset_index()
df

,id,precio,fecha,ranking_capitalizacion,capitalizacion,volumen
0,binancecoin,647.968898,2021-12,4,35526343000,4.616511e+09
1,bitcoin,58612.920116,2021-12,1,556198646739,6.317296e+10
2,cardano,1.741638,2021-12,8,9692632392,3.904049e+09
3,dogecoin,0.219739,2021-12,9,9551718598,6.266509e+09
4,ethereum,4770.314003,2021-12,2,215867156856,4.750751e+10
5,litecoin,216.237271,2021-12,14,5547635550,5.400568e+09
6,matic-network,2.904751,2021-12,13,5778114074,4.947343e+09
7,polkadot,38.205825,2021-12,12,6006607279,2.691884e+09
8,ripple,1.015637,2021-12,5,31134566160,6.329989e+09
9,shiba-inu,0.000048,2021-12,15,5412777834,9.082599e+09


Se realiza una operación de agrupación en el DataFrame `"df"`, agrupando por el id de la criptomoneda. Se aplica una serie de funciones de agregación a las columnas, como el `precio máximo`, la `fecha`, el `ranking mínimo de capitalización`, la `capitalización máxima` y el `volumen máximo`. El resultado se almacena en un nuevo DataFrame llamado `"df"`.

------------

In [956]:
df.to_csv(f"../data/{anio}_{mes}.csv")

Se guarda el DataFrame df en un archivo csv, utilizando el año y el mes proporcionados en las variables anio y mes para nombrar el archivo.